In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

from utils import normalize,toy_data,norm_embed,nmi_score

import matplotlib.pyplot as plt 

from sklearn.utils import shuffle

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from layers import GraphConvolution, InnerProduct
from utils import norm_embed
import torch

In [3]:
import pandas as pd
import networkx as nx

In [4]:
class GNN(nn.Module):
    
    def __init__(self, batch_size, nfeat, nhid, ndim, mu0, sigma0, fixed):
        super(GNN, self).__init__()

        self.gc1 = GraphConvolution(batch_size, nfeat, nhid, mu0, sigma0, scale=False)
        self.fixed = fixed
        if self.fixed:
            self.embeddings = GraphConvolution(batch_size, nhid, 2*ndim, mu0, sigma0, scale=True)
            self.reconstructions = InnerProduct(2*ndim)
        else:
            self.embeddings = GraphConvolution(batch_size, nhid, 4 * ndim, mu0, sigma0, scale=True)
            self.reconstructions = InnerProduct(4*ndim)
        

    def forward(self, x, adj):
        x = self.gc1(x, adj)

        if self.fixed:
            mu = F.relu(self.reconstructions(x))
            return mu, x
        else:
            lr1, lr2 = torch.chunk(x, chunks=2, dim=2)
            mu = F.relu(self.reconstructions(lr1))
            sigma = F.relu(self.reconstructions(lr2))
            return mu, sigma, x
    

In [5]:
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False,precision=8)

In [6]:
# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=426, help='Random seed.')
parser.add_argument('--epochs', type=int, default=500,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.0001,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=10e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--ndim', type=int, default=2,
                    help='Embeddings dimension.')

args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
def load_data(cite=False,cora=False,pub=False):
    
    if cite:
        df1 = pd.read_csv('data/citeseer/citeseer.edges')
        df2 = pd.read_csv('data/citeseer/citeseer.node_labels')
        G = nx.from_pandas_edgelist(df1, 'u', 'v','weight',create_using=nx.DiGraph())
        adj_list = np.array([nx.adjacency_matrix(G).todense()], dtype=float)
        labels = df2.to_numpy()[:,-1].reshape(-1,1)
        labels -= 1
    
    elif cora:
        df1 = pd.read_csv('data/cora/cora.edges')
        df2 = pd.read_csv('data/cora/cora.node_labels')
        G = nx.from_pandas_edgelist(df1, 'u', 'v','weight',create_using=nx.DiGraph())
        adj_list = np.array([nx.adjacency_matrix(G).todense()], dtype=float)
        labels = df2.to_numpy()[:,-1].reshape(-1,1)
        labels -= 1
    
    elif pub:
        df1 = pd.read_csv('data/pubmed/pubmed.edges')
        df2 = pd.read_csv('data/pubmed/pubmed.node_labels')
        G = nx.from_pandas_edgelist(df1, 'u', 'v',create_using=nx.DiGraph())
        adj_list = np.array([nx.adjacency_matrix(G).todense()], dtype=float)
        
    return adj_list, labels

In [8]:
adj, labels = load_data(cite=True)

In [9]:
def svdApprox(adj,dim,relu=False):
    adj = torch.FloatTensor(adj[0])
    U, S, Vh = torch.linalg.svd(adj)
    mu = torch.matmul(torch.matmul(U[:, :dim], torch.diag(S[:dim])), Vh[:dim, :])
    
    embedx = torch.matmul(U[:, :dim],torch.diag(torch.pow(S[:dim], 0.5)))
    embedy = torch.transpose(torch.matmul(torch.diag(torch.pow(S[:dim], 0.5)),Vh[:dim, :]),0,1)
    
    criterion = torch.nn.GaussianNLLLoss()
    if relu:
        crt = torch.nn.ReLU()
        mu = crt(mu)
    mse = torch.nn.MSELoss()
    mseloss = mse(torch.flatten(mu), torch.flatten(adj))
    sig = torch.sqrt(mseloss)
    sigma = sig * torch.ones(adj.shape)
    loss = criterion(torch.flatten(adj), torch.flatten(mu), torch.flatten(torch.square(sigma)))
    
    return mu,sigma,loss.item(),embedx,embedy

In [10]:
def GraphNeuralNet(adj,dim,fixed=False,new=True,features=None,sig_fix=None):
    
    # Set the random seed
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)
        
    args.ndim = dim
    
    adj_norm = normalize(adj)

    adj = torch.FloatTensor(np.array(adj)).cuda()
    
    # loss function
    criterion = torch.nn.GaussianNLLLoss()
    
    # NULL Model
    mu0 = adj.mean()*torch.ones(adj.shape[1:]).cuda()
    sigma0 = adj.std()*torch.ones(adj.shape[1:]).cuda()
    with torch.no_grad():
        loss0 = criterion(torch.flatten(adj), torch.flatten(mu0), torch.flatten(torch.square(sigma0)))
    
    if new:
        if fixed:
            #svd features
            svd_mu,svd_sig,svd_loss,svdembedx,svdembedy = svdApprox(adj=adj.cpu(),dim=dim)
            features = torch.cat((svdembedx,svdembedy),dim=1)
        if not fixed:
            sig_flex = torch.ones(features.shape).cuda()*torch.sqrt(sig_fix/dim).cuda()
            features = torch.cat((features,sig_flex),dim=1).cuda()
        features = features.unsqueeze(dim=0)

    
    

    # Model and optimizer  
        
    model = GNN(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                nhid=adj.shape[1],
                ndim=args.ndim,
                mu0=adj.mean(),
                sigma0=adj.std(),
                fixed=fixed)

    if args.cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()


    # Train model
    t_total = time.time()
    
    # NULL Model
    mu0 = adj.mean()*torch.ones(adj.shape[1:]).cuda()
    sigma0 = adj.std()*torch.ones(adj.shape[1:]).cuda()
    with torch.no_grad():
        loss0 = criterion(torch.flatten(adj), torch.flatten(mu0), torch.flatten(torch.square(sigma0)))

    optimizer = optim.Adam(model.parameters(),
                           lr=args.lr, weight_decay=args.weight_decay)
    
    

    for epoch in range(args.epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()
        
        if fixed:
            mu,lr = model(features, adj_norm)
            with torch.no_grad():
                mse = torch.nn.MSELoss()
                mseloss = mse(torch.flatten(mu),torch.flatten(adj))
                sig = torch.sqrt(mseloss)
            sigma = sig * torch.ones(adj.shape,requires_grad=True).cuda()
        else:
            mu,sigma,lr = model(features, adj_norm)
        
        
        loss = criterion(torch.flatten(adj), torch.flatten(mu), torch.flatten(torch.square(sigma))) 
        loss.backward()
        optimizer.step()

        if epoch == 0:
            best_loss = loss
            best_lr = lr
            if fixed:
                best_sig = sig
        else:
            if loss < best_loss:
                best_loss = loss
                best_lr = lr
                if fixed:
                    best_sig = sig

        if epoch == 0 or (epoch+1) % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'loss: {:.8f}'.format(best_loss.item()))
            

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    if fixed:
        return mu,best_loss.item(),loss0,best_lr,best_sig
    else:
        return mu,best_loss.item(),loss0,best_lr


In [11]:
for dim in [128]:
    print("Fixed Sigma dim {}".format(dim))

    mu,loss,loss0,lr,sigma = GraphNeuralNet(adj=adj,dim=dim,fixed=True)
    mu,loss,loss0,lr,sigma = GraphNeuralNet(adj=adj,dim=dim,fixed=True,new=False,features=lr.detach())

    print("Flexible Sigma dim {}".format(dim))
    args.lr *= 0.1

    mu,loss,loss0,lr = GraphNeuralNet(adj=adj,dim=dim,new=True,features=lr[0].detach(),sig_fix=sigma)
    mu,loss,loss0,lr = GraphNeuralNet(adj=adj,dim=dim,new=False,features=lr.detach())


Fixed Sigma dim 128
Epoch: 0001 loss: -3.64226055
Epoch: 0100 loss: -4.09593439
Epoch: 0200 loss: -4.62287331
Epoch: 0300 loss: -4.62826586
Epoch: 0400 loss: -4.63008642
Epoch: 0500 loss: -4.63103914
Optimization Finished!
Total time elapsed: 122.2137s
Epoch: 0001 loss: -4.63103914
Epoch: 0100 loss: -4.63937473
Epoch: 0200 loss: -4.63939285
Epoch: 0300 loss: -4.63939285
Epoch: 0400 loss: -4.63939285
Epoch: 0500 loss: -4.63939285
Optimization Finished!
Total time elapsed: 123.7603s
Flexible Sigma dim 128
Epoch: 0001 loss: -4.63938665
Epoch: 0100 loss: -6.78978777
Epoch: 0200 loss: -6.79345655
Epoch: 0300 loss: -6.79434299
Epoch: 0400 loss: -6.79513073
Epoch: 0500 loss: -6.79591084
Optimization Finished!
Total time elapsed: 124.0386s
Epoch: 0001 loss: -6.79591084
Epoch: 0100 loss: -6.79861546
Epoch: 0200 loss: -6.80047035
Epoch: 0300 loss: -6.80162811
Epoch: 0400 loss: -6.80244064
Epoch: 0500 loss: -6.80295801
Optimization Finished!
Total time elapsed: 122.5540s


In [12]:
lr1, lr2 = torch.chunk(lr, chunks=2, dim=2)

In [13]:
# set Train %

train_percentage = .8
    
# Train set
number_of_rows = lr1.shape[1]
train_indices = np.random.choice(number_of_rows, size=int(train_percentage*number_of_rows), replace=False)
val_indices = np.setdiff1d(np.arange(lr1.shape[1]),train_indices)

In [14]:
class classify_net(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(classify_net, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Sequential(
            nn.BatchNorm1d(hidden_dim), 
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Sigmoid()
        )
        self.layer3 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        x = self.layer1(x)
        
        for i in range(4):
            x1 = self.layer2(x)
            x = x + x1
            
        return x

In [15]:
def train(embed,labels,lr):

    embed = torch.FloatTensor(embed)
    labels = torch.FloatTensor(labels)
    
    model = classify_net(embed.shape[1],32,7)

    if args.cuda:
        model.cuda()
        embed = embed.cuda()
        labels = labels.cuda()
    

    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr)
    

    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(100000):

        t = time.time()
        
        model.train()

        optimizer.zero_grad()

        output = model(embed)

        train_output = output[train_indices,:]
        train_labels = labels[train_indices,:]
        
        train_accuracy = torch.sum(torch.argmax(train_output,axis=1)==train_labels.reshape(1,-1))/train_labels.shape[0]

        loss = criterion(output,labels.reshape(-1).long())
        loss.backward(retain_graph=True)
        optimizer.step()
        
        model.eval()
        
        # Calculate Validation accuracy
        with torch.no_grad():
            val_output = output[val_indices,:]
            val_labels = labels[val_indices,:]
            val_accuracy = torch.sum(torch.argmax(val_output,axis=1)==val_labels.reshape(1,-1))/val_labels.shape[0]

        # Print summary of training 
        if epoch == 0:
            best_loss = loss
            best_output = output
            best_acc = train_accuracy
            best_val_acc = val_accuracy
            best_val_output = val_output
        else:
            if loss < best_loss:
                best_loss = loss
                best_output = output
                best_acc = train_accuracy
                best_val_acc = val_accuracy
                best_val_output = val_output

        if epoch == 0 or (epoch+1) % 1000 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Train Accuracy: {:.4f}'.format(best_acc.item()),
                  'Validation Accuracy: {:.4f}'.format(best_val_acc.item()),
                  'Loss: {:.8f}'.format(best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))
            
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return 1

In [16]:
train(lr1[0].cpu().detach(),labels,0.001)

Epoch: 0001 Train Accuracy: 0.0352 Validation Accuracy: 0.0413 Loss: 3.58457017 time: 0.0091s
Epoch: 1000 Train Accuracy: 0.4910 Validation Accuracy: 0.4946 Loss: 1.28005040 time: 0.0031s
Epoch: 2000 Train Accuracy: 0.6116 Validation Accuracy: 0.6141 Loss: 1.06029606 time: 0.0000s
Epoch: 3000 Train Accuracy: 0.6775 Validation Accuracy: 0.6462 Loss: 0.94986045 time: 0.0011s
Epoch: 4000 Train Accuracy: 0.7246 Validation Accuracy: 0.6953 Loss: 0.87724733 time: 0.0046s
Epoch: 5000 Train Accuracy: 0.7392 Validation Accuracy: 0.7136 Loss: 0.82673550 time: 0.0040s
Epoch: 6000 Train Accuracy: 0.7522 Validation Accuracy: 0.7381 Loss: 0.79148316 time: 0.0091s
Epoch: 7000 Train Accuracy: 0.7675 Validation Accuracy: 0.7504 Loss: 0.76017016 time: 0.0000s
Epoch: 8000 Train Accuracy: 0.7725 Validation Accuracy: 0.7688 Loss: 0.73753893 time: 0.0071s
Epoch: 9000 Train Accuracy: 0.7790 Validation Accuracy: 0.7688 Loss: 0.72166973 time: 0.0011s
Epoch: 10000 Train Accuracy: 0.7867 Validation Accuracy: 0.7

Epoch: 87000 Train Accuracy: 0.8269 Validation Accuracy: 0.8101 Loss: 0.57795364 time: 0.0000s
Epoch: 88000 Train Accuracy: 0.8269 Validation Accuracy: 0.8101 Loss: 0.57795364 time: 0.0000s
Epoch: 89000 Train Accuracy: 0.8269 Validation Accuracy: 0.8101 Loss: 0.57795364 time: 0.0000s
Epoch: 90000 Train Accuracy: 0.8269 Validation Accuracy: 0.8101 Loss: 0.57795364 time: 0.0000s
Epoch: 91000 Train Accuracy: 0.8196 Validation Accuracy: 0.8208 Loss: 0.57706404 time: 0.0000s
Epoch: 92000 Train Accuracy: 0.8196 Validation Accuracy: 0.8208 Loss: 0.57706404 time: 0.0000s
Epoch: 93000 Train Accuracy: 0.8196 Validation Accuracy: 0.8208 Loss: 0.57706404 time: 0.0046s
Epoch: 94000 Train Accuracy: 0.8196 Validation Accuracy: 0.8208 Loss: 0.57706404 time: 0.0000s
Epoch: 95000 Train Accuracy: 0.8196 Validation Accuracy: 0.8208 Loss: 0.57706404 time: 0.0000s
Epoch: 96000 Train Accuracy: 0.8196 Validation Accuracy: 0.8208 Loss: 0.57706404 time: 0.0131s
Epoch: 97000 Train Accuracy: 0.8196 Validation Acc

1